#Pull down data from ``lanl`` MySQL database and create ``articleClasses``

##Description

Two dataframes are created
- df_years -- (true_article_id, list of gss years)
- df_variables -- (true_article_id, variable, variable_type)

Notes
--
- db is currently sitting on AWS

- Command used to dump tables from lanl database on Rhodes to be put on OSDC:
        mysqldump -p -P3307 -u misha --socket=/mnt/ide1/mysql/var/mysql.sock1  lanl gss_question > gss_question.sql
        
# Tables in the ``lanl`` db

gss_corpus
--
This is the core table and contains information about all of the articles linked to GSS variables. Some of these are book chapters and things didn't have access to, but most we do. It also has variables that correspond to other datasets used.

Core variables:

- true_article_id
- author_[variables]
- title
- publication_title
- volume
- year_published
- file name (the PDF, html and/or texts associated with each of the articles examined as it exists in a file: gss_files) 
- 'doc_type',
- 'citation_type',
- 'author_1_prefix',
- 'author_1_last_name',
- 'author_1_first_name',
- 'author_1_middle_name',
 'author_1_suffix',
 'author_2_prefix',
 'author_2_first_name',
 'author_2_last_name',
 'author_2_middle_name',
 'author_2_suffix',
 'author_3_prefix',
 'author_3_first_name',
 'author_3_last_name',
 'author_3_middle_name',
 'author_3_suffix',
- 'title',
 'pages',
 'pub_id',
- 'publication_title',
 'publisher',
 'volume',
- 'year_published',
 'month_published',
 'publisher_country',
 'publisher_city',
 'state',
 'document_name',
 'conference',
 'editor_first_name',
 'editor_mi',
 'editor_last_name',
 'editor_2_first_name',
 'editor_2_mi',
 'editor_2_last_name',
- 'gss_years',
- 'other_datasets',
- 'abstract',
 'citation',
 'brief_type',
 'docket_number',
 'chapter_name',
 'edition',
 'isbn',
- 'variables',
 'report_no',
- 'file_name',
 'file_url',
 'file_id',
 'unique_pub_id',
 'round',
 'batch',
 'coder1',
 'coder2',
 'coder3',
 'coder4',
 'coder5',
 'coder6',
 'posterior_PaperCorrect',
 'posterior_Central',
 'posterior_Analysis',
 'posterior_Approach'

gss_variables
--
gss_variable_codes
--
These describe the variables (and all of the answers/codes associated with each questions and their individual meanings).

gss_variable_links
--
This links the articles in gss_corpus with the variables used within them.

Core variables:
		
- true_article_id
- variable

gss_question
--
These are responses to the survey about each article

gss_variable_ques
--
These are responses to the survey about each variable (in each article)

In [76]:
import pandas as pd
import cPickle as cp
import sys
sys.path.append('../')
import GSSUtility as GU
import seaborn as sns
import MySQLdb
import numpy as np
from numpy import nan
from random import sample # numpy has its own np.random.sample which works differently and overwrites "random.sample"

In [77]:
db = MySQLdb.connect(host='klab.c3se0dtaabmj.us-west-2.rds.amazonaws.com', user='mteplitskiy', passwd="mteplitskiy", db="lanl")
c = db.cursor()

#Get The Variables

In [78]:
c.execute('select * from gss_variable_ques')
# c.fetchall()
df = pd.DataFrame([el for el in c.fetchall()], columns=('userid', 'author_id', 'article_id', 'var_name', 'var_control', \
                                                        'var_central', 'var_dependent', 'var_independent', 'var_dontknow', \
                                                        'var_type_majority'))
del df['author_id']
del df['var_type_majority']
del df['userid']
df.head()

,article_id,var_name,var_control,var_central,var_dependent,var_independent,var_dontknow
0,2952,FEHOME,,,,,True
1,2952,AGE,,True,,,
2,2952,FEHELP,,,,,True
3,2952,REGION,,True,,,
4,2952,RACE,,True,,,


In [79]:
# convert datatypes so that I can add them to get majority
df.var_central = df.var_central.astype(bool)
df.var_control = df.var_control.astype(bool)
df.var_dependent = df.var_dependent.astype(bool)
df.var_independent = df.var_independent.astype(bool)
df.var_dontknow = df.var_dontknow.astype(bool)

In [80]:
# TALLY THE VOTES FOR EACH VARIABLE TYPE
df_variables = df.groupby(('article_id', 'var_name')).sum()
df_variables.head(5)

var_control  var_central  var_dependent  var_independent  \
article_id var_name                                                             
1          MEMFRAT             0            2              0                0   
           MEMPOLIT            1            0              0                0   
           MEMSERV             0            1              1                0   
           MEMUNION            1            0              0                0   
           MEMVET              1            0              0                0   

                     var_dontknow  
article_id var_name                
1          MEMFRAT              0  
           MEMPOLIT             1  
           MEMSERV              0  
           MEMUNION             0  
           MEMVET               1

###Create majority vote for whether variable is indep or dep

In [81]:
# create a custom max function

# from scipy.stats import rankdata

def custom_max(s): # s is a series
    if s['var_dependent'] > s['var_independent'] + s['var_control']:
        return 'var_dependent'
    elif s['var_dependent'] < s['var_independent'] + s['var_control']:
        return 'var_independent'
    else: 
        return nan 
    
df_variables['majority_vote_type'] = df_variables.apply(custom_max, axis=1)

###Create vote for whether variable is central, ``majority_vote_central``

In [82]:
#If at least one person thinks it's central, then it's central
df_variables['majority_vote_central'] = df_variables.var_central.astype(bool)

#Check how many "ties" there are on variable codes

In [83]:
# there is ~1600 articles but ~22,000 rows in df_variables because each article has many variables.
# So it appears nearly every article has a tie.
def is_there_a_tie(s):
    # if there is a legitimate tie
    if s['var_dependent'] > 0 and (s['var_dependent'] == s['var_independent'] 
                                   or s['var_dependent'] == s['var_independent']+s['var_control']):
        return 'tie: dependent == independent'
    elif s['var_dependent'] > s['var_independent']:
        return 'dep > indep'
    elif s['var_dependent'] < s['var_independent']:
        return 'dep < indep'
    elif s['var_dependent'] < s['var_independent'] + s['var_control']:
        return 'dependent < ind + control'
    elif s['var_dependent'] > s['var_independent'] + s['var_control']:
        return 'dependent > ind + control'
    else: 
        return nan     
df_variables['is_there_a_tie'] = df_variables.apply(is_there_a_tie, axis=1)
print df_variables.is_there_a_tie.value_counts()

dep < indep                      9625
dep > indep                      7256
tie: dependent == independent    1738
dependent < ind + control          33
dtype: int64


In [84]:
df_variables[df_variables.majority_vote_type.isnull()][['majority_vote_type', 'is_there_a_tie']].head()

majority_vote_type                 is_there_a_tie
article_id var_name                                                  
1          MEMFRAT                 NaN                            NaN
18         FEHOME                  NaN  tie: dependent == independent
           FEPOL                   NaN  tie: dependent == independent
           FEPRES                  NaN  tie: dependent == independent
22         RELIG                   NaN  tie: dependent == independent

In [86]:
# How many UNIQUE ARTICLES have at least one tie?
df_var_copy = df_variables.copy()
df_var_copy['article_id'] = df_var_copy.index.get_level_values(0)
df_var_copy['tie'] = df_var_copy.apply(is_there_a_tie, axis=1)
ids_with_tie = df_var_copy[df_var_copy.tie == 'tie: dependent == independent'].article_id.unique()
print 'Unique articles with at least one tie:', len(ids_with_tie)


Unique articles with at least one tie: 265


In [88]:
#How many articles will be *gained* by resolving ties?
#df_var_copy[df_var_copy.article_id.isin(ids_with_tie)].majority_vote_type == 'var_dependent'

#Get the GSS Years

In [90]:
c.execute('select true_article_id, gss_years, year_published from gss_corpus')
df_years = pd.DataFrame([el for el in c.fetchall()], columns=['article_id', 'gss_years', 'year_published'])
df_years.index = df_years.article_id
df_years.gss_years = df_years.gss_years.astype(str)

del df_years['article_id']

df_years.sort('year_published', ascending=False).head()

,gss_years,year_published
article_id,,
8962,,2008
1992,,2008
7497,,2008
1519,,2008
8969,,2008


###Pre-process

In [28]:
GSS_YEARS = [1972, 1973, 1974, 1975, 1976, 1977, 1978, 1980, 1982,  
                 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991,
                 1993, 1994, 1996, 1998, 2000, 2002, 2004, 2006, 2008, 2010,
                 2012]

In [29]:
def f(x):  
    x = x.replace(' ', '')
    x = x.replace('"', '')
    x = x.replace('ISSP', '') # not sure if I should do this? but per 'http://publicdata.norc.org:41000/gssbeta/faqs.html#14' it seems ok
    x = x.strip()
    
    if len(x) < 4: return nan
    
#     x = ''.join([ch for ch in x if not ch.isalpha()])
    
    if x == '' or x == '0' or str(x).startswith('ACT'):
        return nan
    
    return x

df_years.gss_years = df_years.gss_years.map(f)

###Now parse the years strings

In [30]:
def clean_years(x):
    
    # skip all the nans
    if x is np.nan: return np.nan
    
    # for articleID, yearsUncleaned in df[df.gss_years.notnull()].itertuples(): # the "notnull()" should avoid all the nan's
    yearsUncleaned = x

    yearsCleaned = []    # for each article, start with an empty list of GSS years used
 
    print 'yearsUncleaned:', yearsUncleaned
   
    yearsUncleaned = yearsUncleaned.lower().strip().replace('"', '') # lower-case, get rid of trailing spaces and get rid of quotes
    
    # CASE 1: beginning of yearsUncleaned is one of those rows where there's a textual explanation, which seems to always begin with "ACT:" 
    #treat it as if we have no info (don't even put it in the dictionary)
    if yearsUncleaned[:3] == 'act' : return np.nan
   
   # CASE 2: the entire string is less than 4 characters
    if len(yearsUncleaned) < 4:
        print yearsUncleaned
        return np.nan

    # Putting the commands below in a "try" statement because in some instances the GSS year given is 
    # mistaken, because that year doesn't exist. In such cases, the way the exception is handled
    # is simply with a "continue" statement -- the article is skipped (not added to the dictionary)
    # and we move on to the next one     
    try:         
        # split on commas and strip whitespaces
        yearsSplitOnCommas = [e.strip() for e in yearsUncleaned.split(',')]
        for rangeOfYears in yearsSplitOnCommas:
            # Is there a dash? Three formats: 1991-1992 OR 1991-92 OR 1991-2
            if '-' in rangeOfYears:
                yearsSplitOnDash = rangeOfYears.split('-')
    
                # error check:
                if len(yearsSplitOnDash[0]) != 4:
                    print yearsSplitOnDash
                    return np.nan 
                    
                startOfRange = yearsSplitOnDash[0] # the first 4 numbers
                charsToAdd = 4 - len(yearsSplitOnDash[1]) 
                endOfRange = startOfRange[:charsToAdd] + yearsSplitOnDash[1] # characters from beginning of range added to end of range
                
                startIndex = GSS_YEARS.index(int(startOfRange))
                endIndex = GSS_YEARS.index(int(endOfRange))
                
                yearsCleaned.extend( GSS_YEARS[startIndex:endIndex+1] )
            else: # no dash
                # is the string in appropriate year range?
                if int(rangeOfYears) not in GSS_YEARS:
                    print 'rangeOfYears not in GSS_YEARS', rangeOfYears
                    return np.nan 
                else:
                    yearsCleaned.append(int(rangeOfYears))                
        
        return yearsCleaned
    
    except ValueError: # if there is any kind of error regarding the year of the article not being in the available years (GSS_YEARS), skip this article
        print 'This year not in legitimate GSS years', yearsUncleaned
        return np.nan
    
df_years['years_cleaned'] = df_years.gss_years.apply(clean_years)    

yearsUncleaned: 1974-94
yearsUncleaned: 1993
yearsUncleaned: 1993-1998
yearsUncleaned: 1973,1974
yearsUncleaned: 1973,1974,1976,1977
yearsUncleaned: 1972-97
This year not in legitimate GSS years 1972-97
yearsUncleaned: 1989-1991,1996
yearsUncleaned: 1989-1990REINTERVIEW
This year not in legitimate GSS years 1989-1990reinterview
yearsUncleaned: 1973-1978,1980,1982
yearsUncleaned: 1972-78
yearsUncleaned: 1973
yearsUncleaned: 1972
yearsUncleaned: 1976
yearsUncleaned: 1972
yearsUncleaned: 1972
yearsUncleaned: 1972-1994
yearsUncleaned: 1977-1996
yearsUncleaned: 1987,1992
rangeOfYears not in GSS_YEARS 1992
yearsUncleaned: 1991
yearsUncleaned: 1987,1992GSSReinterview
This year not in legitimate GSS years 1987,1992gssreinterview
yearsUncleaned: 1973-1990
yearsUncleaned: 1990
yearsUncleaned: 1972-1983
yearsUncleaned: 1973,1974,1975
yearsUncleaned: 1973,1977
yearsUncleaned: 1973-1988
yearsUncleaned: 1972,1973,1974,1976,1977,1980
yearsUncleaned: 1972-1998
yearsUncleaned: 1972-85
yearsUncleaned: 1

##Now create articleClasses list

In [57]:
'''
description: 
 - This script constructs a list of articleClass instances, where each articleClass contains an article's metadata
 - the filtering of the articles is done mostly by filterArticleClasses.py,
 - But I do make the following filters/selections here:
   
     NOT DOING THIS --> 1. gss_central_variable field must be == 'Yes'
     1. Skip articles with no GSS years used (or if they can't be imputed)
     2. make sure the stated GSS years the article used actually contain the variables
         the article allegedly used

- the other filter criteria are performed by filterArticleClasses.py    
 
 
inputs:
 - I pull things from the lanl database, but also use VARS_BY_YEAR.pickle

outputs:
 - articleClasses.pickle -- this is a list of articleClass instances

'''

# IMPORTS #############################
# add GSS Project/Code directory to module search path, just in case
import sys
sys.path.append('../Code/')
from GSSUtility import articleClass
import cPickle as cp

# GLOBALS
GSS_YEARS = [1972, 1973, 1974, 1975, 1976, 1977, 1978, 
            1980, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 
            1990, 1991, 1993, 1994, 1996, 1998, 
            2000, 2002, 2004, 2006, 2008, 2010, 2012]

pathToData = '../Data/'  
   
# LOAD DATA ###########################
VARS_BY_YEAR = cp.load(open(pathToData + 'VARS_BY_YEAR.pickle', 'rb')) # key=year, value=set('VAR1', 'VAR2', ...)

maxYearPublished = 0
countOfNoGSSYearsUsed = 0
countImputed = 0

# CONSTRUCT articleCLasses LIST
articleClasses = []
article_ids = df_variables.index.levels[0]
for article_id in article_ids: # for each article for which we have information on its variables...
    
    # if articleID not in GSS_CENTRAL_VARIABLE or not GSS_CENTRAL_VARIABLE[articleID]:continue
    
    # sub-dataframe just for this article (original is a multiindex (article, variable name)). 
    # article_df has rows like (var name, metadata columns)
    article_df = df_variables.loc[article_id]
    
    ### VARIABLES
    # get all variable types for the article 
    IVs =  article_df[article_df.majority_vote_type == 'var_independent'].index
    DVs =  article_df[article_df.majority_vote_type == 'var_dependent'].index
    # currently the line below is uesless, because majority_vote_type is never var_control
    controls =  article_df[article_df.majority_vote_type == 'var_control'].index
    centralIVs =  article_df[np.array(article_df.majority_vote_type == 'var_independent') \
                             & np.array(article_df.majority_vote_central == True)].index

    # uncomment later!
#     make sure there is at least one DV and at least one IV
    if len(DVs) == 0 or len(IVs) == 0: continue
        
    # convert all variable names to upper case
    IVs = map(str.upper, IVs)
    DVs = map(str.upper, DVs)
    controls = map(str.upper, controls)
    centralIVs = map(str.upper, centralIVs)
    
    
    ### YEARS
    # this function is used to impute GSSYearsUsed for articles for which we have variable information but not 
    # GSS_years_used information
    def impute_GSS_years_used(variables, publication_year):
        candidate_years = [yr for yr in GSS_YEARS if yr <= (publication_year-2)]
        return [yr for yr in candidate_years if set(IVs + DVs + controls + centralIVs).issubset(VARS_BY_YEAR[yr])]

    yearPublished = df_years.loc[article_id, 'year_published']
    if yearPublished < 1972 or yearPublished > 2014: yearPublished=nan

    # what is this for??
    if yearPublished > maxYearPublished:
        maxYearPublished = yearPublished


    oldGSSYears = df_years.loc[article_id, 'years_cleaned']

    # impute years
    if oldGSSYears is nan:
        if yearPublished is nan:
            countOfNoGSSYearsUsed+=1
            continue
        else:
            # impute GSS years
            oldGSSYears = impute_GSS_years_used(set(IVs + DVs + controls + centralIVs), yearPublished)
            countImputed+=1
            if len(oldGSSYears) == 0: 
                countOfNoGSSYearsUsed+=1
                continue

    # check to make SURE that the GSS years the article allegedly used contain all the VARIABLES
    # the article allegedly used
    oldGSSYears = [yr for yr in oldGSSYears if set(IVs + DVs + controls + centralIVs).issubset(VARS_BY_YEAR[yr])]   

    unusedGSSYears = set(GSS_YEARS) - set(oldGSSYears) # whether the variables are in that year or not..
    newGSSYears = [yr for yr in sorted(unusedGSSYears) if set(IVs + DVs + controls + centralIVs).issubset(VARS_BY_YEAR[yr])]  

    currentArticle = articleClass(article_id, IVs, DVs, controls, centralIVs, oldGSSYears, newGSSYears, yearPublished=yearPublished)
    articleClasses.append(currentArticle)

In [58]:
# save the list    
cp.dump(articleClasses, open(pathToData + 'articleClasses.pickle', 'wb'))

#Let's examine articleClasses to see how much of it is usable!

In [54]:
print 'Total instances:', len(articleClasses)
print 'Skipped articles:', countOfNoGSSYearsUsed
print 'Imputed GSS years:', countImputed
print 'Max year published:', maxYearPublished

Total instances: 875
Skipped articles: 6
Imputed GSS years: 38
Max year published: 2005


##Why no years above 2005, even when imputing?

In [14]:
# we don't have variables information on any of them!!!
# (or year information, either)
articles_above_2005 = list(df_years[df_years.year_published > 2005].index)
print articles_above_2005

for a in articles_above_2005:
    try:
        print df_variables.loc[a]
    except:
        pass

[151, 152, 256, 257, 304, 513, 543, 652, 711, 799, 818, 825, 873, 964, 1086, 1237, 1335, 1519, 1777, 1830, 1879, 1896, 1904, 1905, 1992, 1995, 1998, 2012, 2057, 2259, 2268, 2301, 2371, 2386, 2484, 2494, 2683, 2684, 2718, 2791, 2807, 2929, 2942, 3329, 3657, 3671, 3865, 3933, 3941, 3943, 4070, 4071, 4200, 4201, 4202, 4214, 4358, 4367, 4373, 4375, 4465, 4628, 4689, 4723, 4826, 4853, 4861, 4865, 4918, 4920, 5005, 5120, 5176, 5281, 5334, 5338, 5503, 5537, 5600, 5660, 5745, 5801, 5808, 5824, 5852, 5933, 5966, 6000, 6030, 6064, 6110, 6146, 6206, 6322, 6367, 6368, 6384, 6391, 6398, 6560, 6607, 6694, 6790, 6898, 6900, 7058, 7158, 7160, 7165, 7170, 7182, 7218, 7219, 7357, 7470, 7497, 7665, 7666, 7670, 7754, 7797, 7887, 7936, 8076, 8082, 8092, 8355, 8481, 8520, 8551, 8596, 8637, 8638, 8704, 8720, 8726, 8729, 8735, 8744, 8755, 8756, 8757, 8758, 8759, 8760, 8765, 8772, 8773, 8787, 8788, 8789, 8792, 8793, 8817, 8824, 8827, 8839, 8842, 8845, 8854, 8857, 8865, 8868, 8870, 8892, 8895, 8898, 8931, 8935,

###How many articles don't have a DV and at least one IV?

In [15]:
countNoDvs = 0
countNoIvs = 0
no_dvs = []
no_ivs = []
for a in articleClasses:
    if len(a.DVs) == 0: 
        countNoDvs+=1
        no_dvs.append(a.articleID)
    if len(a.IVs) == 0: 
        countNoIvs+=1
        no_ivs.append(a.articleID)
print 'no IVs', countNoIvs
print 'no DVs', countNoDvs 

no IVs 527
no DVs 583


###Take a look at some of those without DVs

In [16]:
df_variables.loc[no_dvs]

var_control  var_central  var_dependent  var_independent  \
article_id var_name                                                             
23         PREMARSX            0            1              0                0   
           XMARSEX             0            1              0                0   
24         AGE                 0            1              0                2   
           CAPPUN2             0            2              1                1   
           EDUC                0            1              0                2   
           RACE                0            1              0                1   
           SEX                 0            1              0                2   
35         CONARMY             0            1              1                1   
           CONBUS              0            2              1                1   
           CONCLERG            0            1              1                1   
           CONEDUC             0            1              1                1   
           CONFED              0            1              1                1   
           CONJUDGE            0            1              1                1   
           CONLABOR            0            1              1                1   
           CONLEGIS            0            1              1                1   
           CONMEDIC            0            1              1                1   
           CONPRESS            0            1              1                1   
           CONSCI              0            1              1                1   
           CONTV               0            1              1                1   
39         NATARMS             0            1              0                1   
           NATEDUC             0            1              0                1   
           NATENVIR            0            1              0                1   
           NATHEAL             0            1              0                1   
50         PARTYID             0            2              0                0   
73         DISCAFF             0            1              0                0   
           DISCWHY1            0            1              0                0   
           DISCWHY2            0            1              0                0   
           DISCWHY3            0            1              0                0   
           DISCWHY4            0            1              0                0   
           DISCWHY5            0            1              0                0   
...                          ...          ...            ...              ...   
7652       MEMOTHER            1            2              2                1   
           MEMPOLIT            1            2              2                1   
           MEMPROF             1            2              2                1   
           MEMSCHL             1            2              2                1   
           MEMSERV             1            2              2                1   
           MEMSPORT            1            2              2                1   
           MEMUNION            1            2              2                1   
           MEMVET              1            2              2                1   
           MEMYOUTH            1            2              2                1   
           RACE                1            3              0                3   
           RELIG               1            3              0                3   
           VOTE72              0            3              1                2   
7727       PRESTIGE            0            1              0                1   
7738       RACDIN              0            2              0                1   
           RACE                0            2              0                0   
           RACMAR              0            2              0                1   
           RACPRES             0            2       